In [1]:
import numpy as np
import pandas as pd
import os
import re
import tensorflow as tf
from threading import Thread
import time
from tqdm import tqdm
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.offline import init_notebook_mode
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import Sequence
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Activation, Dropout, Flatten, Dense, Input, Layer
from tensorflow.keras.applications import VGG16, ResNet50, DenseNet201, Xception
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

init_notebook_mode(connected=True)

In [2]:
# install Kaggle
!pip install -q kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"chaimabouab","key":"4c9dba2231ee394879b8af8eafc26cdc"}'}

In [3]:
#Creat a kaggle folder
!mkdir ~/.kaggle
#copy the kaggle.json to folder created
!cp kaggle.json  ~/.kaggle/
#permission for the json to act
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d paramaggarwal/fashion-product-images-dataset


100% 23.1G/23.1G [03:28<00:00, 119MB/s]
100% 23.1G/23.1G [03:28<00:00, 119MB/s]


In [4]:
# unzip
!unzip fashion-product-images-dataset.zip

Streaming output truncated to the last 5000 lines.
  inflating: fashion-dataset/styles/58128.json  
  inflating: fashion-dataset/styles/58129.json  
  inflating: fashion-dataset/styles/5813.json  
  inflating: fashion-dataset/styles/58131.json  
  inflating: fashion-dataset/styles/58132.json  
  inflating: fashion-dataset/styles/58133.json  
  inflating: fashion-dataset/styles/58135.json  
  inflating: fashion-dataset/styles/58136.json  
  inflating: fashion-dataset/styles/58137.json  
  inflating: fashion-dataset/styles/58138.json  
  inflating: fashion-dataset/styles/58139.json  
  inflating: fashion-dataset/styles/5814.json  
  inflating: fashion-dataset/styles/58140.json  
  inflating: fashion-dataset/styles/58141.json  
  inflating: fashion-dataset/styles/58143.json  
  inflating: fashion-dataset/styles/58144.json  
  inflating: fashion-dataset/styles/58145.json  
  inflating: fashion-dataset/styles/58146.json  
  inflating: fashion-dataset/styles/58147.json  
  inflating: fashion

In [6]:
images_df = pd.read_csv("./fashion-dataset/fashion-dataset/images.csv")
styles_df = pd.read_csv("./fashion-dataset/fashion-dataset/styles.csv", on_bad_lines='skip')

In [8]:
images_df['id'] = images_df['filename'].apply(lambda x: x.replace(".jpg","")).astype(int)
images_df

,filename,link,id
0,15970.jpg,http://assets.myntassets.com/v1/images/style/p...,15970
1,39386.jpg,http://assets.myntassets.com/v1/images/style/p...,39386
2,59263.jpg,http://assets.myntassets.com/v1/images/style/p...,59263
3,21379.jpg,http://assets.myntassets.com/v1/images/style/p...,21379
4,53759.jpg,http://assets.myntassets.com/v1/images/style/p...,53759
...,...,...,...
44441,17036.jpg,http://assets.myntassets.com/v1/images/style/p...,17036
44442,6461.jpg,http://assets.myntassets.com/v1/images/style/p...,6461
44443,18842.jpg,http://assets.myntassets.com/v1/images/style/p...,18842
44444,46694.jpg,http://assets.myntassets.com/v1/images/style/p...,46694


In [9]:
data = styles_df.merge(images_df,on='id',how='left').reset_index(drop=True)
data['filename'] = data['filename'].apply(lambda x: os.path.join("../input/fashion-product-images-dataset/fashion-dataset/images/",x))
image_files = os.listdir("./fashion-dataset/fashion-dataset/images")

In [10]:
data['file_found'] = data['id'].apply(lambda x: f"{x}.jpg" in image_files)
data = data[data['file_found']].reset_index(drop=True)
data.head()

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,filename,link,file_found
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt,../input/fashion-product-images-dataset/fashio...,http://assets.myntassets.com/v1/images/style/p...,True
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans,../input/fashion-product-images-dataset/fashio...,http://assets.myntassets.com/v1/images/style/p...,True
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016.0,Casual,Titan Women Silver Watch,../input/fashion-product-images-dataset/fashio...,http://assets.myntassets.com/v1/images/style/p...,True
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011.0,Casual,Manchester United Men Solid Black Track Pants,../input/fashion-product-images-dataset/fashio...,http://assets.myntassets.com/v1/images/style/p...,True
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,Puma Men Grey T-shirt,../input/fashion-product-images-dataset/fashio...,http://assets.myntassets.com/v1/images/style/p...,True


In [11]:
data.isnull().sum()

id                      0
gender                  0
masterCategory          0
subCategory             0
articleType             0
baseColour             15
season                 21
year                    1
usage                 317
productDisplayName      7
filename                0
link                    0
file_found              0
dtype: int64

In [12]:
data.drop(columns=['productDisplayName','link','file_found'],inplace=True)
data

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,filename
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,../input/fashion-product-images-dataset/fashio...
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,../input/fashion-product-images-dataset/fashio...
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016.0,Casual,../input/fashion-product-images-dataset/fashio...
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011.0,Casual,../input/fashion-product-images-dataset/fashio...
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,../input/fashion-product-images-dataset/fashio...
...,...,...,...,...,...,...,...,...,...,...
44414,17036,Men,Footwear,Shoes,Casual Shoes,White,Summer,2013.0,Casual,../input/fashion-product-images-dataset/fashio...
44415,6461,Men,Footwear,Flip Flops,Flip Flops,Red,Summer,2011.0,Casual,../input/fashion-product-images-dataset/fashio...
44416,18842,Men,Apparel,Topwear,Tshirts,Blue,Fall,2011.0,Casual,../input/fashion-product-images-dataset/fashio...
44417,46694,Women,Personal Care,Fragrance,Perfume and Body Mist,Blue,Spring,2017.0,Casual,../input/fashion-product-images-dataset/fashio...


In [13]:
import pandas as pd

# Your DataFrame
data['filename'] = data['filename'].apply(lambda x: x.replace('../input/fashion-product-images-dataset/fashion-dataset', './fashion-dataset/fashion-dataset'))

# Print the updated DataFrame
print(data.head())


      id gender masterCategory subCategory  articleType baseColour  season  \
0  15970    Men        Apparel     Topwear       Shirts  Navy Blue    Fall   
1  39386    Men        Apparel  Bottomwear        Jeans       Blue  Summer   
2  59263  Women    Accessories     Watches      Watches     Silver  Winter   
3  21379    Men        Apparel  Bottomwear  Track Pants      Black    Fall   
4  53759    Men        Apparel     Topwear      Tshirts       Grey  Summer   

     year   usage                                           filename  
0  2011.0  Casual  ./fashion-dataset/fashion-dataset/images/15970...  
1  2012.0  Casual  ./fashion-dataset/fashion-dataset/images/39386...  
2  2016.0  Casual  ./fashion-dataset/fashion-dataset/images/59263...  
3  2011.0  Casual  ./fashion-dataset/fashion-dataset/images/21379...  
4  2012.0  Casual  ./fashion-dataset/fashion-dataset/images/53759...  


In [14]:
data.head()

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,filename
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,./fashion-dataset/fashion-dataset/images/15970...
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,./fashion-dataset/fashion-dataset/images/39386...
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016.0,Casual,./fashion-dataset/fashion-dataset/images/59263...
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011.0,Casual,./fashion-dataset/fashion-dataset/images/21379...
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,./fashion-dataset/fashion-dataset/images/53759...


In [15]:
data = data.sample(frac=1).reset_index(drop=True)
n = len(data)
train = data.iloc[:int(n*0.8),:]
val = data.iloc[int(n*0.8):,:].reset_index(drop=True)
train

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,filename
0,56483,Women,Personal Care,Nails,Nail Polish,Brown,Spring,2017.0,NaN,./fashion-dataset/fashion-dataset/images/56483...
1,46362,Men,Apparel,Topwear,Shirts,Maroon,Summer,2012.0,Casual,./fashion-dataset/fashion-dataset/images/46362...
2,36269,Women,Apparel,Loungewear and Nightwear,Night suits,White,Summer,2017.0,Casual,./fashion-dataset/fashion-dataset/images/36269...
3,23621,Girls,Apparel,Dress,Dresses,Navy Blue,Fall,2011.0,Casual,./fashion-dataset/fashion-dataset/images/23621...
4,35720,Men,Footwear,Sandal,Sandals,Black,Summer,2012.0,Casual,./fashion-dataset/fashion-dataset/images/35720...
...,...,...,...,...,...,...,...,...,...,...
35530,43176,Men,Accessories,Watches,Watches,Black,Winter,2016.0,Casual,./fashion-dataset/fashion-dataset/images/43176...
35531,32386,Women,Apparel,Topwear,Tshirts,White,Summer,2012.0,Casual,./fashion-dataset/fashion-dataset/images/32386...
35532,58474,Men,Accessories,Wallets,Wallets,Olive,Winter,2016.0,Casual,./fashion-dataset/fashion-dataset/images/58474...
35533,4256,Men,Apparel,Topwear,Tshirts,Grey Melange,Summer,2011.0,Casual,./fashion-dataset/fashion-dataset/images/4256.jpg


In [16]:
columns_to_drop = ['id', 'gender', 'masterCategory', 'articleType', 'baseColour', 'season', 'year', 'usage']
train2 = train.drop(columns=columns_to_drop)
val2 = val.drop(columns=columns_to_drop)
train2

,subCategory,filename
0,Nails,./fashion-dataset/fashion-dataset/images/56483...
1,Topwear,./fashion-dataset/fashion-dataset/images/46362...
2,Loungewear and Nightwear,./fashion-dataset/fashion-dataset/images/36269...
3,Dress,./fashion-dataset/fashion-dataset/images/23621...
4,Sandal,./fashion-dataset/fashion-dataset/images/35720...
...,...,...
35530,Watches,./fashion-dataset/fashion-dataset/images/43176...
35531,Topwear,./fashion-dataset/fashion-dataset/images/32386...
35532,Wallets,./fashion-dataset/fashion-dataset/images/58474...
35533,Topwear,./fashion-dataset/fashion-dataset/images/4256.jpg


In [17]:
datagen = ImageDataGenerator(rescale=1/255.)

train_generator = datagen.flow_from_dataframe(dataframe=train2,
                                             target_size=(224,224),
                                             x_col='filename',
                                             y_col='subCategory',  # Specify the target column
                                             class_mode='categorical',
                                             batch_size=64,
                                             shuffle=False,
                                             classes=data['subCategory'].unique().tolist())

val_generator = datagen.flow_from_dataframe(dataframe=val2,
                                           target_size=(224,224),
                                           x_col='filename',
                                           y_col='subCategory',  # Specify the target column
                                           class_mode='categorical',
                                           batch_size=64,
                                           shuffle=False,
                                           classes=data['subCategory'].unique().tolist()
                                           )


Found 35535 validated image filenames belonging to 45 classes.
Found 8884 validated image filenames belonging to 45 classes.


In [18]:
print("Length of train generator:", len(train_generator))
print("Length of val generator:", len(val_generator))


Length of train generator: 556
Length of val generator: 139


In [19]:
classes_train = train2['subCategory'].unique().tolist()
num_classes = len(classes_train)
print("Number of unique classes:", num_classes)

Number of unique classes: 44


In [20]:
classes_data = data['subCategory'].unique().tolist()
num_classes_data = len(classes_data)
print("Number of unique classes:", num_classes_data)

Number of unique classes: 45


In [21]:
from keras.applications import Xception
from keras.layers import Dense, Flatten, BatchNormalization, Dropout
from keras.models import Model

def build_model_Xception(name):
    base_model = Xception(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = Flatten()(x)
    x = Dense(4096, activation='relu')(x)  # You can use 'relu' instead of 'leaky_relu' for Xception
    x = BatchNormalization()(x)
    x = Dropout(0.4)(x)
    x = Dense(1024, activation='sigmoid')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.4)(x)
    predictions = Dense(num_classes_data, activation='softmax')(x)

    model = Model(name=name, inputs=base_model.input, outputs=predictions)

    return model


In [22]:
import time
NAME = "Xception-{}".format(int(time.time()))
model_Xception = build_model_Xception(NAME)
model_Xception.summary()
lr = 0.01
epochs = 10
model_Xception.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=lr),
              metrics=['accuracy'])


83683744/83683744 [==============================] - 0s 0us/step
Model: "Xception-1704657893"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 block1_conv1 (Conv2D)       (None, 111, 111, 32)         864       ['input_1[0][0]']             
                                                                                                  
 block1_conv1_bn (BatchNorm  (None, 111, 111, 32)         128       ['block1_conv1[0][0]']        
 alization)                                                                                       
                                                                                                  
 block1_conv1_a

In [ ]:
history_Xception = model_Xception.fit_generator(train_generator,
                    validation_data = val_generator,
                    steps_per_epoch = train_generator.n//train_generator.batch_size,
                    validation_steps = val_generator.n//val_generator.batch_size,
                    epochs=epochs,
                    )

<ipython-input-23-8e3f34380b46>:1: UserWarning:

`Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.



Epoch 1/10
  2/555 [..............................] - ETA: 2:16:34 - loss: 4.2043 - accuracy: 0.3594

In [ ]:
fig = plt.figure(figsize=(10,10))

# Plot accuracy
plt.subplot(221)
plt.plot(history_Xception.history['accuracy'],'bo-', label = "acc")
plt.plot(history_Xception.history['val_accuracy'], 'ro-', label = "val_acc")
plt.title("train_accuracy vs val_accuracy")
plt.ylabel("accuracy")
plt.xlabel("epochs")
plt.grid(True)
plt.legend()

# Plot loss function
plt.subplot(222)
plt.plot(history_Xception.history['loss'],'bo-', label = "loss")
plt.plot(history_Xception.history['val_loss'], 'ro-', label = "val_loss")
plt.title("train_loss vs val_loss")
plt.ylabel("loss")
plt.xlabel("epochs")
plt.grid(True)
plt.legend()


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Generate predictions
y_pred = model_Xception.predict(val_generator)

# Convert predictions and true labels to categorical labels
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = val_generator.classes

# Compute confusion matrix
conf_mat = confusion_matrix(y_true_classes, y_pred_classes)

# Plot the confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(conf_mat, annot=True, fmt='d', cmap='Blues', xticklabels=val_generator.class_indices, yticklabels=val_generator.class_indices)
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()

# Print classification report
#print(classification_report(y_true_classes, y_pred_classes, target_names=val_generator.class_indices.keys()))


In [ ]:
# result of Train Set
score_Xception_train = model_Xception.evaluate_generator(train_generator)
print('Train loss:', score_Xception_train[0])
print('tain accuracy:', score_Xception_train[1])

In [ ]:
# result of Train Set
score_Xception_test = model_Xception.evaluate_generator(val_generator)
print('Train loss:', score_Xception_test[0])
print('tain accuracy:', score_Xception_test[1])